# Examples of using the model inspection class and the profiling functions

The inspection class can gather model parameters, gradients, activations and activation gradients.

The profiling module provide information about cpu and GPU timing, memory usage and flops operations.

In [8]:
import torch
from torchtrainer.models.resunet import ResUNet
from torchtrainer.inspector import Inspector
from torchtrainer.profiling import benchmark_model

# Dummy data. In a real application this should be a batch of the dataset.
batch = torch.rand(8, 1, 224, 224)
labels = torch.randint(0, 1000, (8,))

model = ResUNet((3, 3, 3), (16, 32, 64))
model.eval();

### Model inspection

In [10]:
# Check intermediate activations of the model, as well as gradients and parameters
insp = Inspector(model)
# For activations, we need to explicitly start tracking to set up the forward hooks.
insp.start_tracking_activations()

# Apply model to batch
res = model(batch)
# Dummy calculation of loss and gradients, just as an example.
loss = res.sum().backward()

# Remove the forward hooks
insp.stop_tracking_activations()

# Activations
acts = insp.get_activations()
# Parameters
params = insp.get_params()
# Gradients
grads = insp.get_grads()

names, values = zip(*acts)
print(names[:10])    # Names of the first 10 layers
print(values[0].shape)  # Size of the activations of the first layer

('resunet.conv1', 'resunet.bn1', 'resunet.encoder.stage_0.0.conv1', 'resunet.encoder.stage_0.0.bn1', 'resunet.encoder.stage_0.0.conv2', 'resunet.encoder.stage_0.0.bn2', 'resunet.encoder.stage_0.0.downsample.0', 'resunet.encoder.stage_0.0.downsample.1', 'resunet.encoder.stage_0.0', 'resunet.encoder.stage_0.1.conv1')
torch.Size([8, 16, 224, 224])


Tracking activations involves copying all the data from the GPU to the CPU (to preserve GPU memory), which is expensive. We can provide and aggregation function that will be applied to the data before copying

In [4]:
def agg_func(data, module_name, data_type):
    return torch.tensor([data.min(), data.max()])

insp = Inspector(model, agg_func=agg_func)
insp.start_tracking_activations()
res = model(batch)
insp.stop_tracking_activations()

acts = insp.get_activations()
print(acts)

[('resunet.conv1', tensor([-0.7139,  0.8401])), ('resunet.bn1', tensor([-0.7139,  0.8401])), ('resunet.encoder.stage_0.0.conv1', tensor([-0.7861,  0.7654])), ('resunet.encoder.stage_0.0.bn1', tensor([-0.7861,  0.7654])), ('resunet.encoder.stage_0.0.conv2', tensor([-0.9031,  0.8212])), ('resunet.encoder.stage_0.0.bn2', tensor([-0.9031,  0.8212])), ('resunet.encoder.stage_0.0.downsample.0', tensor([-1.2285,  0.6469])), ('resunet.encoder.stage_0.0.downsample.1', tensor([-1.2285,  0.6469])), ('resunet.encoder.stage_0.0', tensor([0.0000, 0.9857])), ('resunet.encoder.stage_0.1.conv1', tensor([-0.7222,  0.7009])), ('resunet.encoder.stage_0.1.bn1', tensor([-0.7222,  0.7009])), ('resunet.encoder.stage_0.1.conv2', tensor([-0.6484,  0.6623])), ('resunet.encoder.stage_0.1.bn2', tensor([-0.6484,  0.6623])), ('resunet.encoder.stage_0.1', tensor([0.0000, 1.2762])), ('resunet.encoder.stage_0.2.conv1', tensor([-1.3137,  1.0437])), ('resunet.encoder.stage_0.2.bn1', tensor([-1.3137,  1.0437])), ('resunet

We can also only track individual modules

In [5]:
insp = Inspector(model, [model.conv1,model.mid_block], agg_func)
insp.start_tracking_activations()
res = model(batch)
insp.stop_tracking_activations()

acts = insp.get_activations()
print(acts)

[('resunet.conv1', tensor([-0.7139,  0.8401])), ('resunet.mid_block', tensor([ 0.0000, 18.2513]))]


It is also possible to track activation gradients, but it is important to note that they do not work for a layer if the previous layer has an inplace operation.

In [6]:
insp = Inspector(model, [model.conv1])
insp.start_tracking_act_grads()
res = model(batch)
insp.stop_tracking_act_grads()
loss = res.sum().backward()

act_grads = insp.get_act_grads()

### Model profiling

In [7]:
tensor_shape = (8, 1, 224, 224)
# Benchmark the model for training
stats_train = benchmark_model(model, tensor_shape, no_grad=False, call_backward=True, use_float16=True, return_model_info=True)
# Benchmark for inference
stats_val = benchmark_model(model, tensor_shape, no_grad=True, call_backward=False, use_float16=True, return_model_info=False)
# The units for each metric are also included in the dictionary
print(stats_train)
print(stats_val)

Unsupported operator aten::batch_norm encountered 47 time(s)
Unsupported operator aten::add_ encountered 19 time(s)
Unsupported operator aten::upsample_nearest2d encountered 3 time(s)
Unsupported operator aten::add_ encountered 19 time(s)


{'params': 0.450128, 'activations': 0.090116096, 'flops': 15.10699008, 'memory': 0.39917755126953125, 'time_cpu': 0.029001474380493164, 'time_gpu': 0.028121088027954103, 'info': ['params: M', 'activations: G', 'flops: G', 'memory: GiB', 'time_cpu: s', 'time_gpu: s']}
{'params': 0.0, 'activations': 0.0, 'flops': 0.0, 'memory': 0.15419673919677734, 'time_cpu': 0.012001514434814453, 'time_gpu': 0.010200063705444335, 'info': ['params: M', 'activations: G', 'flops: G', 'memory: GiB', 'time_cpu: s', 'time_gpu: s']}
